In [ ]:
from datetime import UTC, datetime

import cabaret

In [ ]:
# Define the observatory components

site = cabaret.Site(
    sky_background=150,  # e-/m^2/arcsec^2/s
    seeing=1.5,  # arcseconds
    elevation=2500,  # meters
    latitude=None,  # degrees
    longitude=None,  # degrees
    sun_altitude=None,  # degrees
)

telescope = cabaret.Telescope(
    focal_length=8,  # meters
    diameter=1.0,  # meters
    collecting_area=None,  # calculated from diameter
)

camera = cabaret.Camera(
    name="Example Camera",
    width=2048,  # pixels
    height=2048,  # pixels
    bin_x=1,  # pixels
    bin_y=1,  # pixels
    pitch=10,  # microns
    plate_scale=None,  # arcseconds per pixel (calculated from pitch and telescope)
    max_adu=65535,  # ADU
    well_depth=50000,  # electrons
    bias=100,  # ADU
    gain=1,  # electrons per ADU
    read_noise=6.2,  # electrons
    dark_current=0.2,  # electrons per second
    average_quantum_efficiency=0.7,  # fraction
    rotation=0,  # degrees
    # pixel_defects={
    #     "hot": {"type": "constant", "value": 65000, "rate": 0.01, "seed": 42},
    #     "cold": {"type": "constant", "value": 100, "rate": 0.01, "seed": 1},
    #     "qe": {"type": "quantum_efficiency_map", "quantum_efficiency_std": 0.25},
    #     "column": {"type": "column", "value": 0, "rate": 0.05, "dim": 0},
    #     "smear": {"type": "readout_smear", "readout_time": 5, "dim": 0},
    # },
)

focuser = cabaret.Focuser(
    position=10000,  # arbitrary units
    best_position=10000,  # arbitrary units
    scale=100,  # seeing_multiplier = 1 + offset / scale
    max_seeing_multiplier=5,  # seeing_multiplier cannot exceed this value
)

In [ ]:
# Create the observatory from the components

observatory = cabaret.Observatory(
    name="Example Observatory",
    site=site,
    telescope=telescope,
    camera=camera,
    focuser=focuser,
)

In [ ]:
# Generate a simulated image and save it as a FITS file

hdu = observatory.generate_fits_image(
    ra=323.362583,  # degrees
    dec=-0.82325,  # degrees
    exp_time=0.5,  # seconds
    dateobs=datetime.now(UTC),  # datetime object in UTC
    filter_band=cabaret.Filters.G,  # filter band
    seed=42,  # random seed
    n_star_limit=100000,  # maximum number of stars to render,
    fwhm_multiplier=7.0,  # to determine the rendering radius around each star
)

In [ ]:
# Save the FITS file
# hdu.writeto("simulated_image.fits", overwrite=True)

In [ ]:
# Plot the simulated image

cabaret.plot_image(hdu[0].data, contrast=0.25, title="Simulated Image")

# Manipulate sources


In [ ]:
import numpy as np
from astropy.coordinates import SkyCoord

filter_band = cabaret.Filters.G
dateobs = datetime.now(UTC)

center = SkyCoord(ra=323.362583, dec=-0.82325, unit="deg")

table = cabaret.GaiaQuery.query(
    center=center,
    radius=camera.get_fov_radius() * 1.5,
    filter_band=filter_band,
    limit=100000,
    # timeout=60,
)

In [ ]:
table_filt = cabaret.GaiaQuery._apply_proper_motion(table, dateobs).copy()

# Filter the table first, then extract the data
fluxes = table_filt[filter_band.value].value.data
nan_mask = np.isnan(fluxes)
table_filt.remove_rows(nan_mask)

# Update fluxes after removing NaN rows
fluxes = table_filt[filter_band.value].value.data
flux_mask = fluxes > 1e4
table_filt.remove_rows(flux_mask)

# Get the final filtered fluxes
fluxes = table_filt[filter_band.value].value.data

sources = cabaret.Sources.from_arrays(
    ra=table_filt["ra"].value.data,
    dec=table_filt["dec"].value.data,
    fluxes=fluxes,
)

In [ ]:
simulated_image = observatory.generate_image(
    ra=center.ra.deg,
    dec=center.dec.deg,
    exp_time=0.5,
    dateobs=dateobs,
    sources=sources,
)

In [ ]:
cabaret.plot_image(
    simulated_image, contrast=0.25, title="Simulated Image without bright sources"
)